# Obtención detalla actividades (Web Scrapping)
(web Ayuntamiento de Madrid)

[Enlace portal web Ayuntamiento de Madrid Actividades Infantiles](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

Voy a trabajar como ejemplo con la [actividad 'Taller de ajedrez'](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

<img src="../img/AyuntMadrid_actividades_01.png" width="500">

## Extracción actividad ejemplo de página 1

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Se encontraron {len(actividades)} actividades.")
        
        # Obtener la primera actividad para análisis detallado
        if actividades:
            primera_actividad = actividades[0]
            
            # Extraer el enlace de la primera actividad
            elemento_enlace = primera_actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_detalle = elemento_enlace['href']
                if not url_detalle.startswith('http'):
                    url_detalle = "https://www.madrid.es" + url_detalle
                
                print(f"Accediendo a la página de detalle: {url_detalle}")
                
                # Hacer una solicitud a la página de detalle
                response_detalle = requests.get(url_detalle)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Imprimir los resultados
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        print(f"{clave}: {valor}")
                    
                    # Crear DataFrame y guardar CSV
                    df = pd.DataFrame([datos_actividad])
                    directorio = '../data/raw/'
                    df.to_csv(f'{directorio}actividad_ejemplo.csv', index=False, encoding='utf-8-sig')
                    print(f"\nDatos guardados correctamente en '{directorio}actividad_ejemplo.csv'")
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("No se encontró el enlace a la página de detalle.")
        else:
            print("No se encontraron actividades.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Se encontraron 25 actividades.
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: 18:30 a 19:30 horas
fecha: Del miércoles 19 de marzo de 2025 al mi

In [2]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,18:30 a 19:30 horas,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...


## Extracción todas las actividades de una página

He comprobado la extracción para un elemento ejemplo. Voy a realizar un bucle para obtener dichos valores de cada una de las 25 actividades de esa pagina 1, y ponerlo en un dataframe para visualizarlo bien.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d/%m/%Y")

# Crear directorio para guardar el CSV si no existe
directorio = '../data/raw/'

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
        print("Comenzando extracción de datos...\n")
        
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"Procesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_detalle = elemento_enlace['href']
                if not url_detalle.startswith('http'):
                    url_detalle = "https://www.madrid.es" + url_detalle
                
                print(f"  Accediendo a la página de detalle: {url_detalle}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(3)  # Esperar 3 segundos entre solicitudes
                response_detalle = requests.get(url_detalle)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_detalle'] = url_detalle
                        
                    # Mostrar información básica de la actividad extraída
                    print(f"  ✓ Extraída: {datos_actividad['título']}")
                    print(f"    Lugar: {datos_actividad['lugar_nombre']}")
                    print(f"    Edad: {datos_actividad['edad']}")
                    print(f"    Fecha: {datos_actividad['fecha']}")
                    print(f"    Precio: {datos_actividad['precio']}")
                    print()
                    
                    # Añadir los datos de esta actividad a la lista general
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"  ✗ Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("  ✗ No se encontró el enlace a la página de detalle.")
        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df = pd.DataFrame(todas_actividades)
            nombre_archivo = f'{directorio}actividades_pagina1_{fecha_hoy}.csv'
            df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Se encontraron 25 actividades.
Comenzando extracción de datos...

Procesando actividad 1 de 25...
  Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD
  ✓ Extraída: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
    Lugar: Biblioteca Pública Municipal María Lejárraga (Hortaleza)
    Edad: de 6 a 17 años
    Fecha: Del miércoles 19 de marzo de 2025 al miércoles 4 de junio de 2025
    Precio: Gratuito

Procesando actividad 2 de 25...
  Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Manualidades/?vgnextfmt=default&vgnextoid=3da790ec79b45910VgnVCM2000001f4a900aRCRD&vgnextc

In [ ]:
# visualización idéntica al ejemplo
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d/%m/%Y")

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
        
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"\nProcesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_actividad = elemento_enlace['href']
                if not url_actividad.startswith('http'):
                    url_actividad = "https://www.madrid.es" + url_actividad
                
                print(f"Accediendo a la página de detalle: {url_actividad}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(2)  # Esperar 2 segundos entre solicitudes
                response_detalle = requests.get(url_actividad)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # diccionario para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer en el texto general tiny_text  (edad, inscripción, periodicidad, día_días, horario, otros datos)
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar edad - versión más amplia pero simplificada
                        datos_actividad['edad'] = "No especificada"
                        
                        # Buscar primero en los blockquotes
                        for blockquote in tiny_text.find_all('blockquote'):
                            if 'Edad:' in blockquote.text and 'años' in blockquote.text:
                                datos_actividad['edad'] = blockquote.text.strip()
                                break

                        # Si no se encontró en blockquotes, buscar en el contenido general
                        if datos_actividad['edad'] == "No especificada":
                            # Dividir el contenido en posibles frases usando puntuación
                            posibles_frases = re.split(r'[.,;:()\n]', contenido_texto)
                            for frase in posibles_frases:
                                frase_limpia = frase.strip()
                                if 'años' in frase_limpia and (
                                    'edad' in frase_limpia.lower() or 
                                    'niñ' in frase_limpia.lower() or 
                                    'de' in frase_limpia.lower() or 
                                    'a partir' in frase_limpia.lower()
                                ):
                                    datos_actividad['edad'] = frase_limpia
                                    break            
                        
                        # # Buscar inscripción
                        # patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        # datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # EXTRACCIÓN MEJORADA DE INSCRIPCIÓN
                        # Primero buscamos en el texto como antes
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        texto_inscripcion = patron_inscripcion.group(0).strip() if patron_inscripcion else ""

                        # Buscar párrafo que contenga "Inscripción" como título fuerte o en encabezado
                        inscripcion_parrafo = ""
                        for p in tiny_text.find_all(['p']):
                            # Verificar si contiene la palabra "inscr" en cualquier parte del párrafo
                            if re.search(r'inscr', p.text, re.IGNORECASE):
                                inscripcion_parrafo = p.text.strip()
                                break

                        # Buscar en div.info-actividad si existe
                        info_actividad = soup_detalle.find('div', class_='info-actividad')
                        if info_actividad:
                            # Buscar el encabezado de inscripción y luego el párrafo de texto que le sigue
                            inscripcion_header = info_actividad.find('h4', class_='inscripcion')
                            if inscripcion_header and inscripcion_header.find_next('p', class_='text-date'):
                                texto_fecha_inscripcion = inscripcion_header.find_next('p', class_='text-date').text.strip()
                                if texto_fecha_inscripcion:
                                    inscripcion_parrafo = texto_fecha_inscripcion

                        # Asignar el valor final de inscripción según lo que se haya encontrado
                        if texto_inscripcion or inscripcion_parrafo:
                            datos_actividad['inscripción'] = texto_inscripcion
                            if inscripcion_parrafo:
                                if texto_inscripcion:
                                    datos_actividad['inscripción'] += ". " + inscripcion_parrafo
                                else:
                                    datos_actividad['inscripción'] = inscripcion_parrafo
                        else:
                            datos_actividad['inscripción'] = "No especificada"

                        # Periodicidad
                        datos_actividad['periodicidad'] = "No especificada"
                        # Agregar detección de patrones como "un martes al mes", "un jueves cada semana", etc.
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        periodos = {
                            'semana': 'semanal',
                            'mes': 'mensual',
                            'año': 'anual'
                        }

                        for dia in dias_semana:
                            for periodo_texto, periodicidad_valor in periodos.items():
                                if re.search(rf'un\s+{dia}\s+(al|cada)\s+{periodo_texto}', contenido_texto, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = periodicidad_valor
                                    break
                            if datos_actividad['periodicidad'] != "No especificada":
                                break
                            
                        # Buscar periodicidad explicitamente declarada                 
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"

                        # Comprobar primero si hay días de la semana en plural (indicativo de periodicidad semanal)
                        dias_semana_plural = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábados', 'domingos']
                        for dia in dias_semana_plural:
                            # Buscar patrones como "Los martes", "Todos los jueves", etc.
                            if re.search(r'(los|las|todos\s+los|todas\s+las)\s+' + dia, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = "semanal"
                                break

                        # Si no se encontró periodicidad con los días en plural, buscar con los patrones originales
                        if datos_actividad['periodicidad'] == "No especificada":
                            for patron in patrones_periodicidad:
                                if re.search(patron, contenido_texto, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = patron
                                    break
                        
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        # patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        # datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                        
                        # Buscar horario con delimitación más precisa (rango u hora única)
                        datos_actividad['horario'] = "No especificado"

                        # Dividir el texto en frases usando signos de puntuación y saltos de línea
                        fragmentos = re.split(r'[.,;()\n]', contenido_texto)

                        for fragmento in fragmentos:
                            fragmento = fragmento.strip()
                            if 'horas' in fragmento.lower():
                                # Buscar rango de horas o una sola hora con "horas"
                                if re.search(r'\d{1,2}[:.]\d{2}\s*(a|-)\s*\d{1,2}[:.]\d{2}', fragmento) or \
                                re.search(r'(\d{1,2}[:.]\d{2})\s*horas', fragmento, re.IGNORECASE) or \
                                re.search(r'\d{1,2}\s*horas', fragmento, re.IGNORECASE):
                                    datos_actividad['horario'] = fragmento
                                    break
                                
                    # Ahora en la parte de info-actividad,buscamos:
                    # Extraer horario, si antes estuviera vacío
                    if datos_actividad['horario'] == "No especificado" and info_actividad:
                        fragmentos_info = re.split(r'[.,;()\n]', info_actividad.get_text())
                        for fragmento in fragmentos_info:
                            fragmento = fragmento.strip()
                            if 'horas' in fragmento.lower():
                                if re.search(r'\d{1,2}[:.]\d{2}\s*(a|-)\s*\d{1,2}[:.]\d{2}', fragmento) or \
                                re.search(r'\d{1,2}[:.]\d{2}\s*horas', fragmento, re.IGNORECASE) or \
                                re.search(r'\d{1,2}\s*horas', fragmento, re.IGNORECASE):
                                    datos_actividad['horario'] = fragmento
                                    break
                     
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"

                    # Si no se encontró precio o es "No especificado", buscar en tiny-text si hay menciones a euros
                    if datos_actividad['precio'] == "No especificado":
                        tiny_text = soup_detalle.find('div', class_='tiny-text')
                        if tiny_text:
                            # Buscar todos los párrafos que contienen "euro" o "euros"
                            precio_parrafos = []
                            for p in tiny_text.find_all(['p', 'blockquote']):
                                if re.search(r'euro[s]?', p.text, re.IGNORECASE):
                                    precio_parrafos.append(p.text.strip())
                            
                            # Si se encontraron párrafos con precio, combinarlos
                            if precio_parrafos:
                                datos_actividad['precio'] = ". ".join(precio_parrafos)
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_actividad'] = url_actividad
                        
                    # Imprimir los resultados en el formato exacto solicitado
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                            print(f"{clave}: {valor}")
                    
                    # Añadir los datos de esta actividad a la lista general
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("No se encontró el enlace a la página de detalle.")
        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df0 = pd.DataFrame(todas_actividades)
            
            # Definir directorio para guardar el CSV
            directorio = '../data/raw/' # Directorio de guardado de datos

            nombre_archivo = f'{directorio}actividades_pagina1_v0_{fecha_hoy}.csv'
            df0.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Se encontraron 25 actividades.

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: Dirigidos a usuarios de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: Las sesiones

In [5]:
df0

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_actividad
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,Dirigidos a usuarios de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,Las sesiones de este taller se celebrarán los ...,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
1,Manualidades,"Taller de manualidades ideado, coordinado y ej...",Dirigido a niñas y niños de 6 a 12 años,El proceso de inscripción finaliza el 23/06/2025,No especificada,martes,de 18 a 19:30 horas,El proceso de inscripción finaliza el 23/06/2025,Biblioteca Pública Municipal Gabriel García Má...,"PLAZA PUEBLO, 2 (CENTRO CULTURAL ORCASUR) 2804...",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
2,Talleres de Price (De 7 a 9 años),¡Diviértete aprendiendo mientras descubres tod...,Edad: De 7 a 9 años,No especificada,No especificada,No especificado,Horario: De 10 a 12 horas,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
3,Talleres del Price (5 y 6 años),¡Diviértete aprendiendo mientras descubres tod...,Edad: 5 y 6 años,No especificada,No especificada,No especificado,No especificado,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",Abono trimestral (10 sesiones): 140 euros\nAbo...,No especificado,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
4,LUDOTECAS,Sábados de abril y laborables no lectivos de S...,para niños y niñas con edad comprendidas entre...,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,21 y 26 de abril de 10:30 a 13:30 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Valverde (Fuencarral - El...,"PLAZA ISLAS AZORES, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
5,TALLERES INFANTILES,Sábados 5 y 19. MONSTRUOS DEL RIO: Para menore...,Para menores de 6 a 8 años,No especificada,No especificada,"sábado, domingo",No especificado,Del sábado 5 de abril de 2025 al domingo 27 de...,Centro de Interpretación de la Naturaleza Mont...,"Carretera M-607 Km 13 L-10, null MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
6,Taller infantil. Reciclo y utilizo,Reciclado creativo para pequeños ecologistas: ...,niños y niñas de 6 a 12 años descubrirán los t...,INSCRIPCIONES: en el centro cultural el primer...,semanal,sábado,de 10:30 a 12 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Cultural Carril del Conde (Hortaleza),"CALLE CARRIL DEL CONDE, 57 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
7,LUDOTECAS,Sábados de abril y días laborales no lectivos ...,para niños y niñas con edades comprendidas ent...,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Alfonso XII (Fuencarral -...,"CALLE MIRA EL RIO, 4 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
8,Deportes populares y tradicionales para familias,Para todos los públicos. Sin inscripción previ...,No especificada,Sin inscripción previa.. Para todos los púb

In [18]:
# codigo original reorganizado y mejorado ChatGPT
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d/%m/%Y")

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    print(f"Se encontraron {len(actividades)} actividades.")
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
       
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"\nProcesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_actividad = elemento_enlace['href']
                if not url_actividad.startswith('http'):
                    url_actividad = "https://www.madrid.es" + url_actividad
                
                print(f"Accediendo a la página de detalle: {url_actividad}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(2)       # Esperar 2 segundos entre solicitudes
                response_detalle = requests.get(url_actividad)

                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Diccionario para almacenar los datos de la actividad
                    datos_actividad = {}

                    contenedor_actividad = soup_detalle.find('div', class_='Panel 1.1')
                    if not contenedor_actividad:
                        print("No se encontró el contenedor de contenido de la actividad.")
                        continue

                    tramites_content = contenedor_actividad.find('div', class_='tramites-content')
                    tiny_text = contenedor_actividad.find('div', class_='tiny-text')
                    # tiny_text = soup_detalle.find('div', class_='tiny-text')  #antes, busca desde la raiz html y puede ser más lento y con errores
                    actividades_info = contenedor_actividad.find('div', class_='actividades-info')
                    info_actividad_fecha_lugar = contenedor_actividad.find('div', class_='info-actividad')

                    # Título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"

                    # Descripción
                    datos_actividad['descripción'] = tiny_text.p.text.strip() if tiny_text and tiny_text.p else "Sin descripción"

                    # Edad
                    # Edad
                    datos_actividad['edad'] = "No especificada"

                    # Función auxiliar para extraer el rango de edad de un texto
                    def extraer_rango_edad(texto):
                        # Patrón para "de X a Y años"
                        patron_rango = re.search(r'de\s+(\d+)\s+a\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_rango:
                            return f"de {patron_rango.group(1)} a {patron_rango.group(2)} años"
                        
                        # Patrón para "Edad: X y Y años" o similares con dos puntos
                        if ':' in texto:
                            partes = texto.split(':', 1)
                            if 'años' in partes[1]:
                                # Extraer solo la parte que contiene la edad después de los dos puntos
                                edad_parte = partes[1].strip()
                                # Si hay texto adicional después de "años", cortarlo
                                if ' años' in edad_parte:
                                    edad_parte = edad_parte[:edad_parte.find(' años') + 5]
                                return edad_parte
                        
                        # Patrón para "entre X y Y años"
                        patron_entre = re.search(r'entre\s+(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_entre:
                            return f"entre {patron_entre.group(1)} y {patron_entre.group(2)} años"

                        # Patrón para "X y Y años" (nomralmente consecutivos)
                        patron_y = re.search(r'(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_y:
                            return f"{patron_y.group(1)} y {patron_y.group(2)} años"
                                                
                        # Patrón para "a partir de X años"
                        patron_a_partir = re.search(r'a\s+partir\s+de\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_a_partir:
                            return f"a partir de {patron_a_partir.group(1)} años"
                        
                        # Si contiene años y alguna palabra clave pero no encaja en patrones anteriores
                        if 'años' in texto and any(palabra in texto.lower() for palabra in ['edad', 'niñ', 'de', 'a partir']):
                            return texto.strip()
                        
                        return None

                    # Buscar en blockquote dentro de tiny_text
                    if tiny_text:
                        for blockquote in tiny_text.find_all('blockquote'):
                            if 'Edad:' in blockquote.text and 'años' in blockquote.text:
                                edad_extraida = extraer_rango_edad(blockquote.text)
                                if edad_extraida:
                                    datos_actividad['edad'] = edad_extraida
                                    break
                        
                        # Si aún no se ha encontrado, buscar en todo el texto de tiny_text
                        if datos_actividad['edad'] == "No especificada":
                            posibles_frases = re.split(r'[.,;:()\n]', tiny_text.get_text())
                            for frase in posibles_frases:
                                edad_extraida = extraer_rango_edad(frase)
                                if edad_extraida:
                                    datos_actividad['edad'] = edad_extraida
                                    break

                    # Buscar en tramites_content si aún no se ha encontrado
                    if datos_actividad['edad'] == "No especificada" and tramites_content:
                        posibles_frases = re.split(r'[.,;:()\n]', tramites_content.get_text())
                        for frase in posibles_frases:
                            edad_extraida = extraer_rango_edad(frase)
                            if edad_extraida:
                                datos_actividad['edad'] = edad_extraida
                                break

                    # Inscripción
                    texto_inscripcion = ""
                    inscripcion_parrafo = ""

                    # Primero buscar en div.info-actividad si existe:
                    if info_actividad_fecha_lugar:
                        h4 = info_actividad_fecha_lugar.find('h4', class_='inscripcion')
                        if h4:
                            siguiente_p = h4.find_next('p', class_='text-date')
                            if siguiente_p:
                                inscripcion_parrafo = siguiente_p.text.strip()

                    # Después buscar también  en tiny_text (independientemente de si ya encontramos algo)
                    if tiny_text:
                        match = re.search(r'([^.]*inscripción[^.]*\.)', tiny_text.get_text(), re.IGNORECASE)
                        if match:
                            texto_inscripcion = match.group(0).strip()
                        
                        # Además, buscar en párrafos específicos dentro de tiny_text
                        for p in tiny_text.find_all('p'):
                            # Verificar si contiene la palabra "inscr" en cualquier parte del párrafo
                            if re.search(r'inscr', p.text, re.IGNORECASE):
                                # Si ya tenemos contenido de info_actividad, concatenamos
                                if texto_inscripcion:
                                    texto_inscripcion += ". " + p.text.strip()
                                else:
                                    texto_inscripcion = p.text.strip()
                                break

                    # Asignar el valor final de inscripción según lo que se haya encontrado
                    if inscripcion_parrafo or texto_inscripcion:
                        # Si tenemos ambos contenidos, los concatenamos
                        if inscripcion_parrafo and texto_inscripcion:
                            datos_actividad['inscripción'] = inscripcion_parrafo + ". " + texto_inscripcion
                        elif inscripcion_parrafo:
                            datos_actividad['inscripción'] = inscripcion_parrafo
                        else:
                            datos_actividad['inscripción'] = texto_inscripcion
                    else:
                        datos_actividad['inscripción'] = "No especificada"

                    # Periodicidad
                    datos_actividad['periodicidad'] = "No especificada"
                    if tiny_text:
                        contenido = tiny_text.get_text()
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        periodos = {'semana': 'semanal', 'mes': 'mensual', 'año': 'anual'}
                        for dia in dias_semana:
                            for periodo, valor in periodos.items():
                                if re.search(rf'un\s+{dia}\s+(al|cada)\s+{periodo}', contenido, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = valor
                                    break
                            if datos_actividad['periodicidad'] != "No especificada":
                                break
                        if datos_actividad['periodicidad'] == "No especificada":
                            patrones = ['diaria', 'semanal', 'quincenal', 'mensual']
                            for patron in patrones:
                                if re.search(patron, contenido, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = patron
                                    break

                    # Día/días
                    datos_actividad['día_días'] = "No especificado"
                    if tiny_text:
                        contenido = tiny_text.get_text()
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        encontrados = [dia for dia in dias_semana if re.search(dia, contenido, re.IGNORECASE)]
                        datos_actividad['día_días'] = ", ".join(encontrados) if encontrados else "No especificado"

                    # Horario
                    datos_actividad['horario'] = "No especificado"
                    if tiny_text:
                        fragmentos = re.split(r'[.,;()\n]', tiny_text.get_text())
                        for frag in fragmentos:
                            if 'horas' in frag.lower() and re.search(r'\d{1,2}[:.]\d{2}', frag):
                                datos_actividad['horario'] = frag.strip()
                                break
                    if datos_actividad['horario'] == "No especificado" and info_actividad_fecha_lugar:
                        fragmentos = re.split(r'[.,;()\n]', info_actividad_fecha_lugar.get_text())
                        for frag in fragmentos:
                            if 'horas' in frag.lower() and re.search(r'\d{1,2}[:.]\d{2}', frag):
                                datos_actividad['horario'] = frag.strip()
                                break

                    # Fecha
                    datos_actividad['fecha'] = "Sin fecha"
                    if info_actividad_fecha_lugar:
                        # Buscar primero el encabezado h4 con clase "fecha title9"
                        fecha_header = info_actividad_fecha_lugar.find('h4', class_='fecha title9')
                        if fecha_header:
                            # Si lo encuentra, buscar el p.text-date inmediatamente después de este encabezado
                            # Esto garantiza que estamos obteniendo la fecha de la sección correcta
                            fecha_elem = fecha_header.find_next('p', class_='text-date')
                            if fecha_elem:
                                datos_actividad['fecha'] = fecha_elem.text.strip()

                    # Lugar nombre
                    datos_actividad['lugar_nombre'] = "Sin lugar"
                    if info_actividad_fecha_lugar:
                        lugar_elem = info_actividad_fecha_lugar.find('a', class_='url fn')
                        if lugar_elem:
                            datos_actividad['lugar_nombre'] = lugar_elem.text.strip()

                    # Dirección
                    datos_actividad['lugar_dirección'] = "Sin dirección"
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        datos_actividad['lugar_dirección'] = re.sub(r'\s+', ' ', direccion)

                    # Precio y recomendación
                    datos_actividad['precio'] = "No especificado"
                    datos_actividad['recomendación'] = "No especificado"
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        if precio_elem:
                            datos_actividad['precio'] = precio_elem.text.strip()
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        if recomendacion_elem:
                            datos_actividad['recomendación'] = recomendacion_elem.text.strip()
                    if datos_actividad['precio'] == "No especificado" and tiny_text:
                        precio_parrafos = [p.text.strip() for p in tiny_text.find_all(['p', 'blockquote']) if 'euro' in p.text.lower()]
                        if precio_parrafos:
                            datos_actividad['precio'] = ". ".join(precio_parrafos)

                    # URL ampliar información
                    datos_actividad['url_ampliar_info'] = "No disponible"
                    if tramites_content:
                        h4 = tramites_content.find('h4', class_='title8', string='Amplíe información')
                        if h4:
                            p = h4.find_next('p')
                            if p and p.find('a'):
                                url_info = p.find('a')['href']
                                datos_actividad['url_ampliar_info'] = "https://www.madrid.es" + url_info if not url_info.startswith('http') else url_info
                    
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_actividad'] = url_actividad
                    
                    # Imprimir los resultados en el formato exacto solicitado
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                            print(f"{clave}: {valor}")
                    
                    todas_actividades.append(datos_actividad)

        if todas_actividades:
            df = pd.DataFrame(todas_actividades)
            directorio = '../data/raw/'
            nombre_archivo = f'{directorio}actividades_pagina1.csv'
            df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"Datos guardados correctamente en '{nombre_archivo}'")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")


Se encontraron 25 actividades.
Contenedor principal encontrado.
Consulta a fecha de 21/04/2025: se encontraron 25 actividades.

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: Las sesiones de es

In [19]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_actividad
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,Las sesiones de este taller se celebrarán los ...,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
1,Manualidades,"Taller de manualidades ideado, coordinado y ej...",de 6 a 12 años,El proceso de inscripción finaliza el 23/06/20...,mensual,martes,de 18 a 19:30 horas,Del martes 25 de marzo de 2025 al martes 24 de...,Biblioteca Pública Municipal Gabriel García Má...,"PLAZA PUEBLO, 2 (CENTRO CULTURAL ORCASUR) 2804...",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
2,Talleres de Price (De 7 a 9 años),¡Diviértete aprendiendo mientras descubres tod...,de 7 a 9 años,No especificada,No especificada,No especificado,No especificado,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
3,Talleres del Price (5 y 6 años),¡Diviértete aprendiendo mientras descubres tod...,5 y 6 años,No especificada,No especificada,No especificado,No especificado,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",Abono trimestral (10 sesiones): 140 euros\nAbo...,No especificado,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
4,LUDOTECAS,Sábados de abril y laborables no lectivos de S...,entre 4 y 10 años,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,21 y 26 de abril de 10:30 a 13:30 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Valverde (Fuencarral - El...,"PLAZA ISLAS AZORES, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
5,TALLERES INFANTILES,Sábados 5 y 19. MONSTRUOS DEL RIO: Para menore...,de 6 a 8 años,No especificada,No especificada,"sábado, domingo",No especificado,Del sábado 5 de abril de 2025 al domingo 27 de...,Centro de Interpretación de la Naturaleza Mont...,"Carretera M-607 Km 13 L-10, null MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
6,Taller infantil. Reciclo y utilizo,Reciclado creativo para pequeños ecologistas: ...,de 6 a 12 años,INSCRIPCIONES: en el centro cultural el primer...,No especificada,sábado,de 10:30 a 12 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Cultural Carril del Conde (Hortaleza),"CALLE CARRIL DEL CONDE, 57 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
7,LUDOTECAS,Sábados de abril y días laborales no lectivos ...,entre 4 y 10 años,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Alfonso XII (Fuencarral -...,"CALLE MIRA EL RIO, 4 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
8,Deportes populares y tradicionales para familias,Para todos los públicos. Sin inscripción previ...,No especificada,Sin inscripción previa.. Para todos los públic...,No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Cultural Galileo (Chamberí),"CALLE GALILEO, 39 MADRID",Gratuito,Recomendado